In [3]:
# General modules
import os
import openai
import pandas as pd
from dotenv import load_dotenv
from helper.utils import *

# Setup API keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key
client = openai.Client()

# Specify models
chat_model_name = 'gpt-4o-mini'
openai_embedding_model = "text-embedding-3-small"
local_embedding_model = "all-MiniLM-L6-v2"

configure_api(client, chat_model_name)

# Specify paths for storing (backup) data
root_dir = r'S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\RivalStars'
data_source = 'Steam'

path_db_prepared = os.path.join(root_dir, data_source, "db_prepared.json")          #backup
path_db_translated = os.path.join(root_dir, data_source, "db_translated.json")      #backup
path_db_analysed = os.path.join(root_dir, data_source, "db_analysed.json")          #backup
path_db_embedded = os.path.join(root_dir, data_source, "db_embedded.json")          #backup
path_db_clustered = os.path.join(root_dir, data_source, "db_clustered.json")        #backup
path_db_final = os.path.join(root_dir, data_source, "db_final.json")                #final file

This Jupyter notebook guides you through the process of generating the sentiment ant topic data for the Rival Stars Desktop edition steam comments.
The process is as follows:
1. Gather Data (Redshift)
2. Translate Data (Optional)
3. Analyse Data
4. Embed Data
5. Cluster Analysis
6. Cluster Naming
7. Upload to Streamlit


# Gather Data (Redshift)

 Currently, we did not automate the process of getting the steam reviews into our Redshift Database. Eventually this will be done so this script already builds on the assumption that the data is already in the database.

In [3]:
# My imports
from helper.redshift_conector_standalone import *

# https://store.steampowered.com/app/1166860/Rival_Stars_Horse_Racing_Desktop_Edition/

# SQL Query Redshift
sql_query = """
SELECT *
FROM steam_review
where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'
"""
logger.info(f"Query Redshift with: {sql_query}")

try:
    results_json, results_df = fetch_query_results(sql_query)
    # Print the first row of the DataFrame
    logger.info("Successfully fetched query results, with shape: %s", results_df.shape)
except Exception as e:
    logger.error(f"Error fetching query results: {e}")
    raise

# Save the json
parsed_json = json.loads(results_json)

# 2) Then pretty-print with indentation
save_to_json(parsed_json, path_db_prepared)

2025-02-11 16:42:05,728 - INFO - Query Redshift with: 
SELECT *
FROM steam_review
where app_id_name = '1166860_Rival_Stars_Horse_Racing_Desktop_Edition'

2025-02-11 16:42:07,828 - INFO - Successfully fetched query results, with shape: (3235, 14)
2025-02-11 16:42:08,014 - INFO - Data successfully saved to S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\RivalStars\Steam\db_prepared.json


# Translate Data (Optional)

Since not all reviews will be in english, we will use a local model to analyse the user review and assign it a language tag.
For the Steam data this step is obsolete, since the data has already a language tag. The function will simply tell you that there is already a language tag.

After this the specified OpenAI model will be used to translate all reviews into english that currently have a different language tag.
The translated review will overwrite the original review in the data, the original tag though will be kept.
Therefore we know the review was translated to check later if the translation caused problems when observing unexpected results.


In [4]:
from helper.data_analysis import *

# Get Language Tag
data = read_json(path_db_prepared)
df = pd.DataFrame(data)
df = detect_language_in_dataframe(df,
                             text_column='review_text',     # which colum to use for language detection
                             language_column='language')    # which column to store the detected language


# Translate the data

df = translate_reviews(df, path_db_translated, id_column='recommendationid',
                       text_column='review_text')  # column with language tag



2025-02-12 09:47:01,467 - INFO - 'language' column already exists. Skipping language detection.
2025-02-12 09:47:01,467 - INFO - No existing file found. Starting fresh.
2025-02-12 09:47:01,467 - INFO - Found 3235 new reviews to process.
2025-02-12 09:47:01,833 - INFO - Translating review ID: 183529438 (Detected Language: spanish)
2025-02-12 09:47:02,990 - INFO - Translating review ID: 179001240 (Detected Language: french)
2025-02-12 09:47:04,466 - INFO - Translating review ID: 174398423 (Detected Language: spanish)
2025-02-12 09:47:05,234 - INFO - Translating review ID: 168385220 (Detected Language: spanish)
2025-02-12 09:47:05,935 - INFO - Translating review ID: 166770178 (Detected Language: unknown)
2025-02-12 09:47:07,603 - INFO - Translating review ID: 158158680 (Detected Language: unknown)
2025-02-12 09:47:08,235 - INFO - Translating review ID: 157559748 (Detected Language: german)
2025-02-12 09:47:09,526 - INFO - Translating review ID: 155242845 (Detected Language: spanish)
2025-

# Analyse Data

A loop iterates over all reviews and will first extract the individual relevant statements from the useres and summarize it under a topic, then this topic will be analysed for its sentiment and if it is a factual statement or a request.

In [16]:
import os
from helper.utils import *
from helper.prompt_templates import *
from helper.data_analysis import normalize_topics_key, process_entry

# Configure API
configure_api(client, chat_model_name)

# currently we store the data sometimes as dataframe and sometimes as JSON. This should be unified at some point.
# For now, we will simply transform the pandas dataframe into a JSON object.
data = pd.read_pickle(path_db_translated)
data_prepared = data.to_dict(orient='records')

id_column = "recommendationid"              # The column that contains unique identifiers
columns_of_interest = ["review_text"]       # The column(s) that are going to be analyzed
all_entries = []
processed_ids = set()

# If the analyzed file already exists, load it
if os.path.exists(path_db_analysed):
    all_entries = read_json(path_db_analysed)
    processed_ids = {entry[id_column] for entry in all_entries}  # set for O(1) membership checks

# Process all unprocessed entries
for i, entry in enumerate(data_prepared):
    current_id = entry[id_column]

    # If we've already processed this entry, skip it
    if current_id in processed_ids:
        logger.info(f"Skipping entry {i} (ID: {current_id}) - already processed.")
        continue

    # Otherwise, process and append
    process_entry(
        entry,
        id_column,
        prompt_template_topic,
        prompt_template_sentiment,
        api_settings,
        columns_of_interest
    )
    all_entries.append(entry)
    processed_ids.add(current_id)  # mark as processed

    # Save intermediate progress every 10 entries
    if (i % 10) == 0 and i != 0:
        save_to_json(all_entries, path_db_analysed)
        logger.info(f"Progress saved at index {i}.")

# Final save after the loop
save_to_json(all_entries, path_db_analysed)
logger.info("All entries processed and final results saved.")

2025-02-12 10:52:08,088 - INFO - Tokens used so far: Prompt Tokens: 0, Completion Tokens: 0
2025-02-12 10:52:08,088 - INFO - Extracting topics for entry ID 183945829
2025-02-12 10:52:09,335 - INFO - Analyzing sentiment for topic 'Visual Appeal' (Entry ID 183945829)
2025-02-12 10:52:09,856 - INFO - Tokens used so far: Prompt Tokens: 749, Completion Tokens: 41
2025-02-12 10:52:09,856 - INFO - Extracting topics for entry ID 183559149
2025-02-12 10:52:11,055 - INFO - Analyzing sentiment for topic 'Overall Enjoyment' (Entry ID 183559149)
2025-02-12 10:52:11,558 - INFO - Tokens used so far: Prompt Tokens: 1507, Completion Tokens: 87
2025-02-12 10:52:11,558 - INFO - Extracting topics for entry ID 183529438
2025-02-12 10:52:12,825 - INFO - Analyzing sentiment for topic 'Overall Enjoyment' (Entry ID 183529438)
2025-02-12 10:52:13,306 - INFO - Tokens used so far: Prompt Tokens: 2258, Completion Tokens: 129
2025-02-12 10:52:13,306 - INFO - Extracting topics for entry ID 180957113
2025-02-12 10:52

KeyboardInterrupt: 

# Embed Data
There are two main ways to embed the data that have been explored here so far. One is a fast local embeding on the generate topic names. It is ~350 dims and can be computed fast. The way slower approch is to use openAI embeddings for not the topics but the actual sentences. This will increase computation time since it is roughly 3500 dimensions per user statement. Especially the API calls will be much slower than the local embedding. The OpenAI embedding though allows for a much mor nuanced semantic representation in the vector which generally improves the results in the clustering step.

### OpenAI Embedding

In [ ]:

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   embedding = client.embeddings.create(input = [text], model=model).data[0].embedding
   return embedding



embed_key = "sentence"  # "topic" or "sentence"
data = read_json(path_db_analysed)

def process_embedding(data, embed_key):
    for i in range(0, len(data)):
        if i % 10 == 0:
            logger.info(f"Processing entry {i}")

        for d_topic in data[i]["topics"]:
            if isinstance(d_topic, dict):
                d_topic["embedding"] = get_embedding(d_topic[embed_key], model="text-embedding-3-small")
    return data


data_embedded = process_embedding(data, embed_key)


# Flatten
def flatten_data(data):
    flattened = []
    for entry in data:
        base_copy = dict(entry)
        topics = base_copy.pop("topics", [])

        for topic in topics:
            new_entry = dict(base_copy)
            new_entry.update(topic)
            flattened.append(new_entry)
    return flattened


data_flattened = flatten_data(data_embedded)

# Save the embedded data
save_to_json(data_flattened, path_db_embedded)

### Local Embedding

In [17]:
from helper.embedding import *

embed_key = "topic"  # "topic" or "sentence"

data = read_json(path_db_analysed)
embed_model = initialize_embedding_model(local_embedding_model)


def process_embedding(data, embed_key):
    for i in range(0, len(data)):
        if i % 10 == 0:
            logger.info(f"Processing entry {i}")

        for d_topic in data[i]["topics"]:
            if isinstance(d_topic, dict):
                d_topic["embedding"] = embed_text(d_topic[embed_key], embed_model)
    return data


data_embedded = process_embedding(data, embed_key)


# Flatten
def flatten_data(data):
    flattened = []
    for entry in data:
        base_copy = dict(entry)
        topics = base_copy.pop("topics", [])

        for topic in topics:
            new_entry = dict(base_copy)
            new_entry.update(topic)
            flattened.append(new_entry)
    return flattened


data_flattened = flatten_data(data_embedded)

# Save the embedded data
save_to_json(data_flattened, path_db_embedded)

2025-02-12 12:18:42,512 - INFO - Loading embedding model: all-MiniLM-L6-v2
S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\helper\embedding.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=model_name))
2025-02-12 12:18:54,251 - INFO - Use pytorch device_name: cpu
2025-02-12 12:18:54,251 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-02-12 12:18:57,301 - INFO - Processing entry 0
2025-02-12 12:18:57,544 - INFO - Processing entry 10
2025-02-12 12:18:57,717 - INFO - Processing entry 20
2025-02-12 12:18:57,947 - INFO - Processin

# Cluster Analysis

The main approach here is to use HDBSCAN to cluster the embeddings. kmeans can additionally be used if the user has a rough idea about the number of topics in the data set. If the openAI embedding was used in the previous step it is generally advised to perform a dimension reduction before clustering. The UMAP algorithm is used for this purpose.
This is because HDBSCAN is dense based. And in a high dimensional space the data is generally sparse, which leads to less meaningful clusters and vastly greater computation time.

### Dimensionality Reduction (Optional)

In [ ]:
# Reduce dimensions a priori
# The clustering does not perform to good. Some datapoints that clearly should be in a cluster based on eyeballing and their topic name but they are not. Rather often some points in a dense cluster are categorized as noise.
# I will try to improve this by first perform a dimension reduction and then perform clustering. Reason being, that in high dimensions the data might be too sparse for the clustering algorithm to work properly.


import umap

data = read_json(path_db_embedded)
# Gather all embeddings
embeddings = [entry['embedding'] for entry in data]

# Convert to numpy array
X = np.array(embeddings)

# Perform UMAP
X_embedded = umap.UMAP(n_components=40).fit_transform(X) # 40 dimensions

# Store the updated embeddings in the data
for i, entry in enumerate(data):
    entry['embedding'] = X_embedded[i].tolist()



### HDBSCAN

In [27]:
from helper.cluster_analysis import *
from helper.utils import *

# Adjustable parameters
dimensionality_methods = ['UMAP', 'PCA', 'tSNE']
hdbscan_params = {"min_cluster_size": 50, "min_samples": 2, "cluster_selection_epsilon": 0.15}

data = read_json(path_db_embedded)
df = pd.DataFrame(data)
df = df[df['embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
print(f"Loaded {len(df)} valid entries with embeddings.")

# Extract embeddings
mat = np.array(df['embedding'].tolist())

hdbscan_clusterer = hdbscan.HDBSCAN(**hdbscan_params)
cluster_labels = hdbscan_clusterer.fit_predict(mat)

reduction_results = {}

for method in dimensionality_methods:
    coords_2d = dimensionality_reduction(mat, method, n_components=2)
    reduction_results[f'hdbscan_{method}_2D'] = {
        'x': coords_2d[:, 0],
        'y': coords_2d[:, 1]
    }

# 3D Reduction
    coords_3d = dimensionality_reduction(mat, method, n_components=3)
    reduction_results[f'hdbscan_{method}_3D'] = {
        'x': coords_3d[:, 0],
        'y': coords_3d[:, 1],
        'z': coords_3d[:, 2]
    }

# Add dimensional coordinates to DataFrame
for method_dim, coords in reduction_results.items():
    for axis, values in coords.items():
        df[f'{method_dim}_{axis}'] = values

# Add the cluster labels to the DataFrame
df['hdbscan_id'] = cluster_labels


Loaded 9501 valid entries with embeddings.


C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-02-12 12:26:39,206 - INFO - Applying UMAP with 2 components.
C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-02-12 12:26:41,260 - INFO - Applying UMAP with 3 components.
C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.wa

### KMeans (Optional)

In [28]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

from helper.cluster_analysis import dimensionality_reduction


# Adjustable parameters
dimensionality_methods = ['UMAP', 'PCA', 'tSNE']
kmeans_clusters = [15, 21, 25, 50]


df = df[df['embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
print(f"Loaded {len(df)} valid entries with embeddings.")

# Extract embeddings
mat = np.array(df['embedding'].tolist())  # shape (n_samples, n_dimensions)

# --- KMeans Clustering (High-Dimensional) ---
for n_clusters in kmeans_clusters:
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans_model.fit_predict(mat)

    logger.info(f"Found {len(np.unique(cluster_labels))} clusters for KMeans with {n_clusters} clusters.")

    # Store the labels in a column named by cluster count
    df[f'kmeans_{n_clusters}_id'] = cluster_labels

# --- Dimensionality Reduction ---
# We do each method in 2D and 3D exactly once
reduction_results = {}

for method in dimensionality_methods:
    # 2D
    coords_2d = dimensionality_reduction(mat, method, n_components=2)
    reduction_results[f'kmeans_{method}_2D'] = {
        'x': coords_2d[:, 0],
        'y': coords_2d[:, 1]
    }

    # 3D
    coords_3d = dimensionality_reduction(mat, method, n_components=3)
    reduction_results[f'kmeans_{method}_3D'] = {
        'x': coords_3d[:, 0],
        'y': coords_3d[:, 1],
        'z': coords_3d[:, 2]
    }

# --- Add Dimensional Coordinates to DataFrame ---
for method_dim, coords in reduction_results.items():
    for axis, values in coords.items():
        df[f'{method_dim}_{axis}'] = values



Loaded 9501 valid entries with embeddings.


2025-02-12 12:27:37,951 - INFO - Found 15 clusters for KMeans with 15 clusters.
2025-02-12 12:27:38,093 - INFO - Found 21 clusters for KMeans with 21 clusters.
2025-02-12 12:27:38,292 - INFO - Found 25 clusters for KMeans with 25 clusters.
2025-02-12 12:27:38,476 - INFO - Found 50 clusters for KMeans with 50 clusters.
2025-02-12 12:27:38,478 - INFO - Applying UMAP with 2 components.
C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-02-12 12:27:41,032 - INFO - Applying UMAP with 3 components.
C:\Users\fbohm\Documents\Venvironments\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-02-12 12:27:43,514 - INFO - Applying PCA with 2 components.
2025-02-12 12:27:43,600 - INFO - Applying PCA w

In [29]:
# Save the clustered data
# NOTE: either cluster algorithm is optional as well as the dimension reduction. Generally you want to perform HDBSCAN though with reduced dimensions and use kmeans only if you have an idea of the number of clusters up front.
save_df_as_json(df, path_db_clustered)

2025-02-12 12:28:41,123 - INFO - Saving data to S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\RivalStars\Steam\db_clustered.json


# Cluster Naming
Lastly to make the clusters more meaningful on first sight, they will be named. Again OpenAIs models will be used to perfomr that. The k most centric points will be computed and the model will generate a name for the cluster based on these points.

In [30]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

from helper.cluster_naming import *

api_settings = {"client": client, "model": chat_model_name}

def name_clusters(
    df,
    cluster_columns,
    embedding_col="embedding",
    text_col="sentence",
    top_k=25,
    skip_noise_label=-1
):
    for col in cluster_columns:
        # Prepare a dict to store {cluster_id -> cluster_name}
        cluster_id_to_name = {}
        logger.info((f'Preparing to name clusters in column "{col}"'))

        # Get unique cluster IDs from this column
        cluster_ids = df[col].unique()

        for cluster_id in cluster_ids:

            if skip_noise_label is not None and cluster_id == skip_noise_label:
                continue

            # Select rows belonging to this cluster
            cluster_data = df[df[col] == cluster_id]
            if cluster_data.empty:
                continue

            # Compute centroid of embeddings
            embeddings = np.array(cluster_data[embedding_col].tolist())
            centroid = embeddings.mean(axis=0, dtype=np.float32, keepdims=True)

            # Find top_k closest points to centroid
            distances = cosine_distances(centroid, embeddings).flatten()
            closest_indices = np.argsort(distances)[:top_k]
            representative_texts = cluster_data.iloc[closest_indices][text_col].tolist()

            # Call your naming function
            cluster_name = generate_cluster_name(representative_texts, api_settings)
            cluster_id_to_name[cluster_id] = cluster_name

        # Create a new column with the cluster name for each row
        name_col = f"{col}_name"
        df[name_col] = df[col].apply(lambda cid: cluster_id_to_name.get(cid, "Noise"))

    return df


data = read_json(path_db_clustered)  # data is probably a list of dicts
df = pd.DataFrame(data)              # Convert to DataFrame

cluster_columns = ['hdbscan_id', 'kmeans_15_id', 'kmeans_21_id', 'kmeans_25_id', 'kmeans_50_id']

df_named = name_clusters(
    df,
    cluster_columns,
    embedding_col="embedding",
    text_col="sentence",
    top_k=10,
    skip_noise_label=-1  # for HDBSCAN noise
)


2025-02-12 12:30:39,139 - INFO - Preparing to name clusters in column "hdbscan_id"
2025-02-12 12:30:39,718 - INFO - Generated cluster name: Game Love and Enthusiasm
2025-02-12 12:30:39,718 - INFO - Tokens used so far: Prompt Tokens: 162, Completion Tokens: 6
2025-02-12 12:30:40,286 - INFO - Generated cluster name: Evening Gaming Enjoyment
2025-02-12 12:30:40,286 - INFO - Tokens used so far: Prompt Tokens: 324, Completion Tokens: 12
2025-02-12 12:30:40,949 - INFO - Generated cluster name: Engaging Horse Management Experience
2025-02-12 12:30:40,964 - INFO - Tokens used so far: Prompt Tokens: 519, Completion Tokens: 18
2025-02-12 12:30:41,633 - INFO - Generated cluster name: Horse Customization and Personalization
2025-02-12 12:30:41,633 - INFO - Tokens used so far: Prompt Tokens: 738, Completion Tokens: 25
2025-02-12 12:30:42,146 - INFO - Generated cluster name: Game Enthusiasm and Praise
2025-02-12 12:30:42,146 - INFO - Tokens used so far: Prompt Tokens: 923, Completion Tokens: 31
2025

In [31]:
# Optionally get rid of the embeddings to save space
df_named.drop(columns=['embedding'], inplace=True)

In [32]:
save_data_for_streamlit(df_named, path_db_final)

2025-02-12 12:32:32,505 - INFO - Saving updated data to S:\SID\Analytics\Working Files\Individual\Florian\Projects\DataScience\cluster_analysis\Data\RivalStars\Steam\db_final.json
2025-02-12 12:32:32,867 - INFO - Data saved successfully.
